<a href="https://colab.research.google.com/github/Komal-patra/RAG/blob/main/2_Query_Transformation_MasterRAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U python-dotenv
!pip install -q -U sentence-transformers
!pip install -q -U faiss-cpu
!pip install -q -U huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.4/436.4 kB 8.8 MB/s eta 0:00:00


In [2]:
!pip install -q -U langchain langchain-community langchain-core langchain-groq # install the langchain package

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.8/401.8 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.5 MB/s eta 0:00:00


In [3]:
import bs4
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain.schema.output_parser import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_groq import ChatGroq
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

### Setting the Environment variables and API keys

In [4]:
from google.colab import userdata
import os

In [5]:
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = userdata.get('LANGSMITH_API_KEY')
os.environ['LANGCHAIN_PROJECT'] = 'The RAG Query Transformation'
os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')

# Multi-Query

- https://python.langchain.com/docs/how_to/MultiQueryRetriever/

In [6]:
loader = WebBaseLoader(
    web_path=("https://lilianweng.github.io/posts/2023-06-23-agent/"),
    bs_kwargs= dict(
        parse_only = bs4.SoupStrainer(
            class_=("post-content","post-title","post-header")
        )
    ),
)

In [7]:
docs = loader.load()

Splitting and Chunking

In [8]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

splits = text_splitter.split_documents(docs)

Embeddings

In [9]:
model_name = "BAAI/bge-small-en"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
hf_embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Store the embeddings into vector store

In [10]:
vectorstore = FAISS.from_documents(documents=splits,
                                    embedding=hf_embeddings)

retriever = vectorstore.as_retriever()

Multi-Query prompt

In [11]:
from langchain.prompts import ChatPromptTemplate

# Multi Query: Different Perspectives
template = """You are an AI language model assistant. Your task is to generate five
different versions of the given user question to retrieve relevant documents from a vector
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search.
Provide these alternative questions separated by newlines. Original question: {question}"""
prompt_perspectives = ChatPromptTemplate.from_template(template)

generate_queries = (
    prompt_perspectives
    | ChatGroq(temperature=0)
    | StrOutputParser()
    | (lambda x: x.split("\n"))
)

In [12]:
generate_queries.invoke("What is Task decomposition?")

['1. Can you explain the concept of task decomposition in a simple way?',
 '2. How does task decomposition work in the context of problem-solving?',
 '3. What are the key steps involved in the process of task decomposition?',
 '4. How can task decomposition improve efficiency in achieving complex goals?',
 '5. Can you provide real-world examples where task decomposition is applied?']

To find the common answer for all the retrieved docs from the multiple query:

- create the function to get the unique union

In [13]:
from langchain.load import dumps,loads

In [14]:
def get_unique_union(documents: list[list]):
  """Unique union of retrieved docs"""
  flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
  unique_docs = list(set(flattened_docs))
  return [loads(doc) for doc in unique_docs]

Retrieve

In [15]:
question = "what is task decomposition for LLM agents?"

retrieval_chain = generate_queries | retriever.map() | get_unique_union
docs = retrieval_chain.invoke({"question": question})

<ipython-input-14-47fb25728f92>:5: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  return [loads(doc) for doc in unique_docs]


In [16]:
len(docs)

8

In [17]:
docs

[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Fig. 2.  Examples of reasoning trajectories for knowledge-intensive tasks (e.g. HotpotQA, FEVER) and decision-making tasks (e.g. AlfWorld Env, WebShop). (Image source: Yao et al. 2023).\nIn both experiments on knowledge-intensive tasks and decision-making tasks, ReAct works better than the Act-only baseline where Thought: … step is removed.\nReflexion (Shinn & Labash 2023) is a framework to equips agents with dynamic memory and self-reflection capabilities to improve reasoning skills. Reflexion has a standard RL setup, in which the reward model provides a simple binary reward and the action space follows the setup in ReAct where the task-specific action space is augmented with language to enable complex reasoning steps. After each action $a_t$, the agent computes a heuristic $h_t$ and optionally may decide to reset the environment to start a new trial depending on the self-reflection re

Generate

In [18]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough


In [19]:
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

In [20]:
prompt = ChatPromptTemplate.from_template(template)

In [21]:
llm = ChatGroq(temperature=0)

final_rag_chain = (
    {"context": retrieval_chain,
     "question": itemgetter("question")}
    | prompt
    | llm
    | StrOutputParser()
)

In [22]:

final_rag_chain.invoke({"question": question})

'Task decomposition for LLM (large language model) agents refers to the process of breaking down large tasks into smaller, manageable subgoals. This allows the agent to handle complex tasks more efficiently. Task decomposition can be done using simple prompting techniques with the LLM, using task-specific instructions, or with human inputs. The goal is to shed light on the model\'s thinking process and transform big tasks into multiple manageable tasks. An example of a task decomposition technique is Chain of Thought (CoT), which instructs the model to "think step by step" to utilize more test-time computation to decompose hard tasks into smaller and simpler steps.'

# RAG Fusion using RRF (Reciprocal Rank Fusion)

In [23]:
from langchain.load import dumps, loads

def reciprocal_rank_fusion(results: list[list], k=60):
    """ Reciprocal_rank_fusion that takes multiple lists of ranked documents
        and an optional parameter k used in the RRF formula """

    # Initialize a dictionary to hold fused scores for each unique document
    fused_scores = {}

    # Iterate through each list of ranked documents
    for docs in results:
        # Iterate through each document in the list, with its rank (position in the list)
        for rank, doc in enumerate(docs):
            # Convert the document to a string format to use as a key (assumes documents can be serialized to JSON)
            doc_str = dumps(doc)
            # If the document is not yet in the fused_scores dictionary, add it with an initial score of 0
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            # Retrieve the current score of the document, if any
            previous_score = fused_scores[doc_str]
            # Update the score of the document using the RRF formula: 1 / (rank + k)
            fused_scores[doc_str] += 1 / (rank + k)

    # Sort the documents based on their fused scores in descending order to get the final reranked results
    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]

    # Return the reranked results as a list of tuples, each containing the document and its fused score
    return reranked_results

retrieval_chain_rag_fusion = generate_queries | retriever.map() | reciprocal_rank_fusion
docs = retrieval_chain_rag_fusion.invoke({"question": question})
len(docs)

8

Function: reciprocal_rank_fusion
1. Input Parameters:
results: This is a list of lists. Each sublist contains ranked documents returned from different retrieval models or sources. For example, it could be the top documents retrieved from various queries.
k: A parameter used in the Reciprocal Rank Fusion formula. It adjusts how quickly the contribution of documents decreases as their rank increases (default value is 60).
2. Fused Scores Initialization:
fused_scores: This is a dictionary that will store each document (converted to a string form for consistency) as the key, and its fused score as the value.
3. Iterating Over Document Lists:
The outer loop goes through each list in results. The inner loop goes through each document within the sublist, treating it as a ranked list.

For each document, the rank is determined by its position in the list (starting at 0), and the RRF score is computed using:

score
+
=
1
rank
+
𝑘
score+=
rank+k
1
​

Documents that appear higher in the list (i.e., with a smaller rank) will get a higher score due to the reciprocal nature of the formula.
4. Serializing the Document:
doc_str = dumps(doc): Each document is serialized (converted to a string) using dumps so that it can be stored as a dictionary key. This assumes the documents can be JSON-encoded.
5. Updating Fused Scores:
The fused score of each document is updated based on its rank across multiple ranked lists. If a document appears in several lists, its score accumulates.
6. Sorting and Reranking:
Once all the documents have been processed and their scores calculated, they are sorted in descending order by their scores using:
python
Copy code
sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
The function then deserializes each document from its string form using loads() and pairs it with its final score in a list of tuples (reranked_results).
7. Returning Reranked Results:
The reranked_results list is returned, where each element is a tuple containing the document and its final fused score.


In [24]:
docs

[(Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Or\n@article{weng2023agent,\n  title   = "LLM-powered Autonomous Agents",\n  author  = "Weng, Lilian",\n  journal = "lilianweng.github.io",\n  year    = "2023",\n  month   = "Jun",\n  url     = "https://lilianweng.github.io/posts/2023-06-23-agent/"\n}\nReferences#\n[1] Wei et al. “Chain of thought prompting elicits reasoning in large language models.” NeurIPS 2022\n[2] Yao et al. “Tree of Thoughts: Dliberate Problem Solving with Large Language Models.” arXiv preprint arXiv:2305.10601 (2023).\n[3] Liu et al. “Chain of Hindsight Aligns Language Models with Feedback\n“ arXiv preprint arXiv:2302.02676 (2023).\n[4] Liu et al. “LLM+P: Empowering Large Language Models with Optimal Planning Proficiency” arXiv preprint arXiv:2304.11477 (2023).\n[5] Yao et al. “ReAct: Synergizing reasoning and acting in language models.” ICLR 2023.\n[6] Google Blog. “Announcing ScaNN: Efficient Vector Similarity

In [25]:
from langchain_core.runnables import RunnablePassthrough

# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    {"context": retrieval_chain_rag_fusion,
     "question": itemgetter("question")}
    | prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"question":question})

'Task decomposition for LLM (large language model) agents refers to the process of breaking down a complex task into smaller, manageable subgoals. This can be achieved through various methods, such as simple prompting techniques like "Steps for XYZ" or "What are the subgoals for achieving XYZ?", using task-specific instructions, or with human inputs. One standard prompting technique for enhancing model performance on complex tasks is chain of thought (CoT), which instructs the model to "think step by step" to utilize more test-time computation and decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and provides insights into the model\'s thinking process. Tree of Thoughts extends CoT by exploring multiple reasoning possibilities at each step, creating a tree structure with each state evaluated by a classifier or majority vote.'